In [1]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt

%pip install corextopic

from corextopic import corextopic as ct
from corextopic import vis_topic as vt 

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


LOAD DATASET HERE 

In [0]:
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))


// THE ROWS ARE DOCUMENTS AND THE COLUMNS ARE COUNT. VECTORINZING(SPARSE)

// ENGLISH SET OF STOP WORDS

// 20,000 WORDS FROM EVERY DOCUMENT

In [3]:
vectorizer = CountVectorizer(stop_words='english', max_features=20000, binary=True)
doc_word = vectorizer.fit_transform(newsgroups.data)
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # DOCUMENTS x WORDS

(11314, 20000)

In [4]:
words = list(np.asarray(vectorizer.get_feature_names()))
not_digit_inds = [ind for ind,word in enumerate(words) if not word.isdigit()]
doc_word = doc_word[:,not_digit_inds]
words    = [word for ind,word in enumerate(words) if not word.isdigit()]

doc_word.shape # DOCUMENT x WORDS


(11314, 19038)

TRAINING A MODEL WITH 50 TOPICS

In [0]:
topic_model = ct.Corex(n_hidden=50, words=words, max_iter=200, verbose=False, seed=1)
topic_model.fit(doc_word, words=words);

PRINTING WORDS IN ONE TOPIC

In [6]:
topic_model.get_topics(topic=1, n_words=5)

[('team', 0.07605118792666486),
 ('game', 0.06666754127977209),
 ('season', 0.04887993842511198),
 ('players', 0.04734426771432984),
 ('league', 0.043708887912550405)]

PRINT ALL WORDS IN ALL TOPICS

In [7]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: dsl,n3jxp,chastity,cadre,geb,shameful,intellect,skepticism,banks,pitt
1: team,game,season,players,league,play,hockey,games,teams,nhl
2: government,law,public,rights,state,encryption,clipper,federal,security,secure
3: god,jesus,bible,christians,christian,christ,religion,jews,church,faith
4: people,say,fact,point,believe,person,saying,world,reason,mean
5: armenians,armenian,national,international,argic,press,policy,serdar,soviet,armenia
6: file,program,window,directory,ftp,pub,server,application,unix,available
7: based,issue,sense,clear,truth,subject,certain,known,particular,existence
8: cs,ma,au,gmt,cc,uu,id,sites,fi,host
9: windows,software,card,thanks,pc,dos,files,disk,advance,ram
10: drive,sale,scsi,controller,board,shipping,ide,drives,cd,bus
11: pitching,hit,staff,braves,runs,hitter,nl,smith,hr,baltimore
12: just,don,like,time,going,right,better,let,come,didn
13: archive,various,document,related,addition,modified,published,contents,complete,distributed
14: information,internet,un

TOTAL CORRELATION SHOULD BE MAX; DONE BY COREX; RUN THE ALGORITHM MULTIPLE TIMES AND STORE THE ONE WITH THE MAXIMUM TC

In [8]:
topic_model.tc

44.547808454659226

// TO GET MORE CONCENTRATED WORDS I COULD ADD ONE MORE COREX MODEL WHICH TAKES THESE TOPICS AND WORDS AS INPUT

// IMPROVING MODEL BY ADDING ANCHORING WORDS

// ADDING ANCHOR WORD NASA FOR THE FIRST TOPIC; ONLY THE FIRST TOPIC GETS REPLACED, REST ARE STILL IN DESCENDING ORDER OF THEIR TC SCORES

In [0]:
anchor_words = ['nasa']

In [0]:
anchored_topic_model =  ct.Corex(n_hidden=50, seed=2)
anchored_topic_model.fit(doc_word, words=words, anchors=anchor_words, anchor_strength=6);


// NOW THE TOPIC BECOMES NASA. ALL THE WORDS RELATED TO NASA ARE PRINTED. USE THIS FOR NRC/CAA SPECIFIC WORDS. WE USE THESE WORDS TO FORM WORD CLOUD

In [11]:
topic_words,_ = zip(*anchored_topic_model.get_topics(topic=0))
print('0: ' + ','.join(topic_words))


0: nasa,gov,ames,institute,jpl,station,propulsion,jsc,arc,shafer


// SET ANCHOR WORDS LIKE THIS TO GET EFFECT OF NRC/CAA FOR ECONIMICS, REVENUE, POLITICS, RURAL



// anchor_words = ['nrc', ['nrc', 'politics'], ['nrc', 'economy'], ['nrc','rural'],['nrc','detection']]

// WE USE THIS TO SEE THE RELATED WORDS OR UNDERLYING TOPICS(WORDS) IGNORED

In [0]:
# anchor_words = ['nrc', ['nrc', 'politics'], ['nrc', 'economy'], ['nrc','rural'],['nrc','detection']]

In [0]:
#for n in range(len(anchor_words)):
    #topic_words,_ = zip(*anchored_topic_model.get_topics(topic=n))
    #print('{}: '.format(n) + ','.join(topic_words))


In [12]:
# vt.vis_rep(topic_model, column_label=words, prefix='twenty')


Print topics in text file
